In [2]:

from bs4 import BeautifulSoup
import bs4
import requests
import time
import pandas as pd
import random
import numpy as np

In [3]:
# A "data junior" search seems to give me jobs that I think are interesting: 
# https://ca.indeed.com/jobs?q=data%20junior&l=Canada
# When you go on the next page it:s
# https://ca.indeed.com/jobs?q=data+junior&l=Canada&start=10
# Then next page:
# https://ca.indeed.com/jobs?q=data+junior&l=Canada&start=20

# There only seems to be about 10 pages of results.
# If you try something like:
# https://ca.indeed.com/jobs?q=data+junior&l=Canada&start=1000
# It just give you the last page, so just check until the response stops changing?

# Also, you can use start=0 for the first page:
# https://ca.indeed.com/jobs?q=data+junior&l=Canada&start=0
# Is the same as
# https://ca.indeed.com/jobs?q=data%20junior&l=Canada

MAX_NUMBER_OF_PAGES_PER_SEARCH = 10 # testing

pages = []
for start in range(0, MAX_NUMBER_OF_PAGES_PER_SEARCH*10, 10):
    page = requests.get('https://ca.indeed.com/jobs?q=data%20junior&l=Canada&start=' + str(start))
    soup = BeautifulSoup(page.text, 'html.parser')
    pages.append(soup)
    time.sleep(random.randrange(3, 6) + random.random())

In [37]:
# from bs4 import BeautifulSoup
# from selenium import webdriver
# driver = webdriver.Chrome()
# driver.set_page_load_timeout(5)
# driver.get('https://ca.indeed.com/jobs?q=data%20junior&l=Canada&start=0')
# html = driver.page_source
# soup = BeautifulSoup(html)

# # check out the docs for the kinds of things you can do with 'find_all'
# # this (untested) snippet should find tags with a specific class ID
# # see: http://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-by-css-class
# for tag in soup.find_all("div", class_="jobsearch-SerpJobCard unifiedRow row result"):
#     print(tag.text)

In [38]:
import re

TEST_INPUT = """<ul style="list-style-type:circle;margin-top: 0px;margin-bottom: 0px;padding-left:20px;">\n<li style="margin-bottom:0px;">Interpret <b>data</b>, formulate reports, and make recommendations to the team.</li>\n<li>Remain fully informed on latest <b>data</b> trends, practice, and process.</li>\n</ul><ul style="list-style-type:circle;margin-top: 0px;margin-bottom: 0px;padding-left:20px;">\n<li style="margin-bottom:0px;">Interpret <b>data</b>, formulate reports, and make recommendations to the team.</li>\n<li>Remain fully informed on latest <b>data</b> trends, practice, and process.</li>\n</ul>"""

def clean_HTML(input: str) -> str:
    """I can't display HTML inside a cell in a DataFrame (it seems)"""
    # remove tags <somethign> and leave contents between tags
    out = re.sub("<.*?>", '', input)
    out = re.sub('\n', ' ', out)
    return out

print(clean_HTML(TEST_INPUT))

 Interpret data, formulate reports, and make recommendations to the team. Remain fully informed on latest data trends, practice, and process.  Interpret data, formulate reports, and make recommendations to the team. Remain fully informed on latest data trends, practice, and process. 


In [59]:
# When you search there's box cards for each posting, the class for the box is:
# class="jobsearch-SerpJobCard unifiedRow row result clickcard"
# that wasn't working for me though, so I changed it to 'class':"job_seen_beacon"

titles = pd.Series([], dtype='string')
companyNames = pd.Series([], dtype='string')
jobSnippets = pd.Series([], dtype='string')
locations = pd.Series([], dtype='string')
dates = pd.Series([], dtype='string')
for page in pages:
    jobs = page.find_all('div', attrs={'class':"job_seen_beacon"})
    for job in jobs:
        
        # Job Title
        title = job.find('h2', attrs={'class':'jobTitle'}).find('span', attrs={'title':True}).next 
        titles = titles.append(pd.Series([str(title)]))

        # Company Name
        companyName = job.find('span', attrs={'class':'companyName'})
        tempCompanyName = companyName
        companyName = companyName.find('a', attrs={'class':'turnstileLink companyOverviewLink'})
        if companyName is None: companyName = tempCompanyName
        companyName = companyName.next       
        companyNames = companyNames.append(pd.Series([str(companyName)]))
        
        # Job snip
        jobSnippet = job.find('div', attrs={'class':'job-snippet'}).next
        jobSnippets = jobSnippets.append(pd.Series([str(clean_HTML(str(jobSnippet)))]))

        # Location
        location = job.find('div', attrs={'class':'companyLocation'}).next
        locations = locations.append(pd.Series([str(location)]))

        # Posting Date
        date = job.find('span', attrs={'class':'date'}).next
        dates = dates.append(pd.Series([str(date)]))
        
pd.set_option('display.max_colwidth', None)
#titles, companyNames, jobSnippets
job


<div class="job_seen_beacon"><table cellpadding="0" cellspacing="0" class="jobCard_mainContent" role="presentation"><tbody><tr><td class="resultContent"><div class="heading4 color-text-primary singleLineTitle tapItem-gutter"><h2 class="jobTitle jobTitle-color-purple"><span title="Data Steward I, ITSS Data Governance">Data Steward I, ITSS Data Governance</span></h2></div><div class="heading6 company_location tapItem-gutter"><pre><span class="companyName"><a class="turnstileLink companyOverviewLink" data-tn-element="companyName" href="/cmp/Td-Bank" rel="noopener" target="_blank">TD Bank</a></span><span class="ratingsDisplay withRatingLink"><a class="ratingLink" data-tn-variant="cmplinktst2" href="/cmp/Td-Bank/reviews" rel="noopener" target="_blank" title="TD Bank reviews"><span aria-label="3.8 of stars rating" class="ratingNumber" role="img"><span aria-hidden="true">3.8</span><svg aria-hidden="true" class="starIcon" fill="none" height="12" role="presentation" viewbox="0 0 16 16" width="1

In [78]:
def clean_date_Series(input: str) -> str:
    """Remove the characters from the days ago posting col
    so I can sort postings by that number."""
    output = re.sub('\D', '', input)
    output = re.sub('\s*', '', output)
    output = int(output)
    return output

TEST_DATE = """23 days ago"""
assert clean_date_Series(TEST_DATE) == 23
clean_dates = dates.apply(clean_date_Series)

In [82]:
assert len(titles) == len(companyNames) == len(jobSnippets) == len(locations) == len(dates) == len(clean_dates)
df = pd.DataFrame({f'{titles=}'.split('=')[0]: titles, f'{companyNames=}'.split('=')[0]: companyNames, f'{jobSnippets=}'.split('=')[0]: jobSnippets, f'{locations=}'.split('=')[0]: locations, f'{dates=}'.split('=')[0]: dates, f'{clean_dates=}'.split('=')[0]: clean_dates})
df = df.reset_index()
df.tail()

,index,titles,companyNames,jobSnippets,locations,dates,clean_dates
136,0,"Analyst, Business I",Mosaic North America,"Interpret data, formulate reports, and make recommendations to the team. Remain fully informed on latest data trends, practice, and process.","Toronto, ON",24 days ago,24
137,0,Laboratory Assistant I - Virology Pre-analytics,Alberta Precision Laboratories,"You may be required to perform data entry and specimen processing functions in other areas of the lab. As a Laboratory Assistant I, Clinical Microbiology …","Edmonton, AB",4 days ago,4
138,0,Business Development Analyst I,Best Buy Canada,"Provides insights to team on key opportunities areas of growth with existing and new categories, through analysis of key market data.","Burnaby, BC",10 days ago,10
139,0,Junior Financial Analyst,The Group Master,"Reporting to the assistant controller, the junior financial analyst is a key player in producing information that helps decision-making.","Boucherville, QC",12 days ago,12
140,0,"Data Steward I, ITSS Data Governance",TD Bank,Complete metadata and data quality tasks. Identify and monitor the quality of critical data elements. Manage data work activities requiring coordination across…,"Toronto, ON",11 days ago,11


In [84]:
sorted_df = df.sort_values(f'{clean_dates=}'.split('=')[0])
sorted_df.head()

,index,titles,companyNames,jobSnippets,locations,dates,clean_dates
27,0,Analyste PMO junior - Junior PMO Analyst,Gameloft,"Being responsible for the regular data analytics, report generation and key performance indicators. 1 year of experience in project management or PMO-related…","Montréal, QC",1 day ago,1
45,0,SAS Developer,MSi Corp (Bell Canada),Document data mapping diagrams and data dictionaries. Work with business analysts to understand the business requirements for data modelling and data context.,"Ottawa, ON",2 days ago,2
60,0,I/O R.O. Enrolment Data Quality Assurance Officer,George Brown College,"Experience with data mining and reporting utilizing current software applications such as Microsoft Access, SQL data mining and reporting is required.","Toronto, ON",2 days ago,2
59,0,Jr. Data Analyst,Flex Surveys,Translating complex data and research outcomes into engaging and actionable formats for clients. We offer an array of survey consulting services and develop…,"Greater Toronto Area, ON",2 days ago,2
4,0,Junior Data Analyst,FlightHub,Analyzing data sets and identifying trends and patterns; Our team will count on you to use data from various sources to identify issues as they happen and to…,"Montréal, QC",2 days ago,2
